# How to Run TARDIS Model Grids

This notebook demonstrates the capabilities of the TARDIS grid. The grid facilitates users running large numbers of TARDIS simulations.

In [ ]:
import pandas as pd
import numpy as np
from tardis.io.atom_data.util import download_atom_data
from tardis.io.atom_data import AtomData

import tardis.grid as grid

In [ ]:
# We download the atomic data needed to run a TARDIS simulation
download_atom_data('kurucz_cd23_chianti_H_He')


## Creating A Grid

There are 2 ways of creating a TARDIS grid: directly from a dataframe, or by defining a set of axes over which the grid should be defined. In both cases, a config.yml file is required. **Note that for the dataframe, the column names are in the form of valid keys in a tardis Configuration dictionary. For the axes, the keys must similarly be valid tardis Configuration keys.**

In [ ]:
# Load a DataFrame
df = pd.read_csv('example_grid.txt')
df

In [ ]:
# Create a tardis grid directly from a dataframe.
grid.tardisGrid(configFile='example.yml', gridFrame=df)

In [ ]:
# Create an axes dictionary
axesdict = {'model.structure.velocity.start':np.arange(10000,15000,1000), 
            'model.abundances.He':np.arange(0,1,0.1),
            'model.abundances.H':np.arange(0,1,0.25)}

In [ ]:
#Create a tardis grid from an axes dict using the classmethod.
grid.tardisGrid.from_axes(configFile='example.yml', axesdict=axesdict)

## TARDIS Grid Attributes

The TARDIS grid only has 2 attributes. It creates a TARDIS Configuration object from the user specified config.yml file and saves this to the `self.config` attribute. The grid also stores the parameters of every grid point in a Dataframe accessed by `self.grid`

In [ ]:
tg = grid.tardisGrid(configFile='example.yml', gridFrame=df)

In [ ]:
# The config generated from the user's yml file.
tg.config

In [ ]:
# The user specified grid.
tg.grid

## TARDIS Grid Functionality

The TARDIS Grid provides a variety of functions for using the grid to generate new Configurations, return new SimulationState objects, or directly run simulations using the parameters specified by the grid. This functionality is particularly useful for running large numbers of simulations and easily works with JobArrays where the row_index is the JobArray index.

In [ ]:
# Easily get a new TARDIS Configuration object 
# which has the properties of the base self.config
# but with properties modified by a row of the grid.
new_grid = tg.grid_row_to_config(row_index=0);
print("tg.config is the original config:",tg.config.model.abundances)
print("The new config is modified: ",new_grid.model.abundances)

In [ ]:
# In case a user needs to make more complicated changes
# to the base TARDIS model (i.e. using parameters that 
# are not valid TARDIS Configuration keys), the grid
# can return a SimulationState object for a given row.
atom_data = AtomData.from_hdf(tg.config.atom_data)
simulation_state = tg.grid_row_to_simulation_state(row_index=0, atomic_data=atom_data)
simulation_state

In [ ]:
# Users can easily run a full TARDIS simulation
# using the grid.
sim = tg.run_sim_from_grid(row_index=0)